## 9.7 Deutsch-Jozsa algorithm
A demonstration of the [Deutsch-Jozsa algorithm](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm).

In [ ]:
using ImageShow
using StrangelyDisplayed
using StrangelyQuantum

The following function creates an oracle acting on `n+1` bits. The argument `f` chooses an oracle matrix:

- if `f` is **false**, the oracle uses an identity matrix. This corresponds to a constant function.
- if `f` is **true**, the oracle uses a matrix that swaps the ancilla qubit when the last input qubit is 1. This corresponds to a balanced function.

In [ ]:
function createOracle(n::Integer, f::Bool)
    dim = 2 << n
    half = dim ÷ 2

    matrix = zeros(ComplexF64, dim, dim)

    if !f
        for i = 1:dim
            matrix[i, i] = 1
        end
        return Oracle(matrix)
    elseif f
        for i = 1:dim
            if isodd(i)
                matrix[i, i] = 1
            else
                if i <= half
                    matrix[i, i + half] = 1
                else
                    matrix[i, i - half] = 1
                end
            end
        end
        return Oracle(matrix)
    end
end

The next cell contains a function that tests the functions created by `createOracle` for `n` qubits. This is a generalisation of the `n == 1` case provided [earlier](ch09-05-deutsch.ipynb).

In [ ]:
function deutsch_josza(n, choice)
    simulator = SimpleQuantumExecutionEnvironment()
    # Create a program with N + 1 qubits. We need N qubits
    # for the input bits and an additional ancilla qubit.
    program = Program(n + 1)
    step0 = Step()
    # Apply a Pauli-X gate to the ancilla qubit
    addGate(step0, X(n + 1))

    step1 = Step()
    # Apply a Hadamard gate to all qubits, bringing
    # them into superposition
    for j = 1:(n + 1)
        addGate(step1, Hadamard(j))
    end

    step2 = Step()
    # Fetch and apply an oracle to the circuit
    oracle = createOracle(n, choice)
    addGate(step2, oracle)

    step3 = Step()
    # Apply a Hadamard gate to all input qubits (not
    # to the ancilla qubit)
    for j = 1:n
        addGate(step3, Hadamard(j))
    end

    addStep(program, step0)
    addStep(program, step1)
    addStep(program, step2)
    addStep(program, step3)
    # Execute the program and measure the first qubit
    result = runProgram(simulator, program)
    qubits = getQubits(result)
    println("f = ", choice, ", val = ", measure(qubits[1]))
    return program
end

Test the algorithm for 3 qubits and a constant function:

In [ ]:
program1 = deutsch_josza(3, false)

In [ ]:
drawProgram(program1)

The measured value of the first qubit is 0, so the oracle applies a **constant** function.

Test the algorithm for 3 qubits and a balanced function:

In [ ]:
program2 = deutsch_josza(3, true)

In [ ]:
drawProgram(program2)

The measured value of the first qubit is 1, so the oracle applies a **balanced** function.